# Deferred Feedback Evaluation

Running feedback in "deferred" mode allows them to be computed by a separate process or even computer as long as it has access to the same database as the tru wrapper. In this notebook we demonstrate how to set this up.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

In [ ]:
import numpy as np
from trulens.core import Feedback
from trulens.core import Select
from trulens.core import Tru
from trulens.core import TruCustomApp
from trulens.core.schema import FeedbackMode
from trulens.ext.provider.huggingface.provider import Dummy
from trulens.utils.threading import TP

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp

tp = TP()

d = Dummy(
    loading_prob=0.0,
    freeze_prob=0.0,
    error_prob=0.0,
    overloaded_prob=0.0,
    rpm=6000,
)

tru = Tru()

tru.reset_database()

tru.start_dashboard(force=True, _dev=Path().cwd().parent.parent.resolve())

In [ ]:
# Set up some feedback functions based on the dummy provider as well as the
# example dummy app.

f_dummy_min = (
    Feedback(
        d.positive_sentiment,
        name="min aggregate",
    )
    .on(text=Select.Record.main_output[::20])
    .aggregate(np.min)
)

f_dummy_max = (
    Feedback(d.positive_sentiment, name="max aggregate")
    .on(text=Select.Record.main_output[::20])
    .aggregate(np.max)
)


# Create custom app:
ca = CustomApp()

In [ ]:
# Create trulens wrapper with the previously defined feedback functions,
# specifying `feedback_mode`.

ta = TruCustomApp(
    ca,
    app_id="customapp",
    feedbacks=[f_dummy_min, f_dummy_max],
    feedback_mode=FeedbackMode.DEFERRED,  # deferred feedback mode
)

In [ ]:
# Run the app. This will not produce any feedbacks but will add them to the
# database for the deferred evaluator to run them later.

with ta as recorder:
    ca.respond_to_query("hello")

In [ ]:
# Start the deferred feedback evaluator. This is a non-blocking call. If you are
# running this in a seperate process, make sure you don't exit.

tru.start_evaluator()